In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from typing import List

### Constantes

In [2]:
DIR_DATASET_DADOS_BNDES: str = os.path.join('datasets', 'dados-originais')
DIR_OUTPUT: str = os.path.join('datasets')

ARQUIVO_INDICE_ATIVIDADE_ECONOMICA = os.path.join(DIR_DATASET_DADOS_BNDES, '0.1 - Exo - IBC-Br.xls')
ARQUIVO_PARAMETROS_ESTRUTURA_TERMO_TX_JUROS = os.path.join(DIR_DATASET_DADOS_BNDES, '0.3 - Exo - ETTJ - progress.xls')
ARQUIVO_INDICES_ONIBUS_CAMINHOES = os.path.join(DIR_DATASET_DADOS_BNDES, '1.1 - Endo - FINAME OeC.xls')
ARQUIVO_OUTPUT_OEC_MPME = os.path.join(DIR_OUTPUT, 'dados-gerados', 'oec_mpme.csv')

DATA_CORRENTE = datetime.today().strftime("%Y-%m-%d")
TAXA_FIXA_TIF = 0.15

### Funções 

In [3]:
def calcula_custo(row: pd.core.series.Series) -> float: 
    """
        O custo de captação varia de acordo com a sua fase, utiizando variávies distintas que foram previamente montadas em uma estrutura padas única.
        A função calcula automaticamente o custo de acordo com o conteúdo da coluna data_base
        
        Args:
            row (pd.core.series.Series): Serie pandas com formato esperado para obtenção dos parâmetros de cálculo
        Returns:
            Custo de captação 
        
    """
    
    #: float, spread: float, tx_tjlp: float, tx_ji: float, tx_inflacao: float, tx_di_60: float, tx_selic: float) -> float :
    data_base: str = row['data_base']
    custo_psi: float = row['custo_psi']
    spread: float = row['spread']
    tx_tjlp: float = row['A_TJLP']
    tx_selic: float = row['tx_fixa_selic']
    tx_di_60: float = row['tx_pre_prazo_medio']
    tx_inflacao: float = row['inflacao']
    tx_ji: float = row['A_Ji']    
    
    # PSI
    if (data_base >= '2009-10-01' and data_base <= '2015-12-01'):
        return custo_psi
    # TJLP
    elif (data_base >= '2016-01-01' and data_base <= '2017-12-01'):
        return spread + tx_tjlp + TAXA_FIXA_TIF
    # TLP
    elif (data_base >= '2018-01-01' and data_base <= '2020-07-01'):
         return ( (1 + (spread + TAXA_FIXA_TIF) / 100) * ( 1 + tx_ji/100) * ( 1 + tx_inflacao/100) - 1) * 100
    # SELIC    
    elif (data_base >= '2020-08-01'):
        return tx_di_60 + tx_selic + TAXA_FIXA_TIF
    else:
         return np.nan

### Exógena - Indice de Atividade Econômica

In [4]:
df_indice_atv_economica = pd.read_excel(ARQUIVO_INDICE_ATIVIDADE_ECONOMICA)
df_indice_atv_economica = df_indice_atv_economica[['A_IBC','Ano','Mês']]
df_indice_atv_economica.rename(columns={'A_IBC': 'ind_atv_econ', 'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_indice_atv_economica = df_indice_atv_economica.dropna()
df_indice_atv_economica['data_base'] = df_indice_atv_economica.ano.map(str) + '-' + df_indice_atv_economica.mes.map("{:02}".format) + '-01'
df_indice_atv_economica = df_indice_atv_economica[(df_indice_atv_economica.data_base >= '2009-10-01') & (df_indice_atv_economica.data_base < DATA_CORRENTE)]


In [5]:
DATA_CORRENTE

'2021-12-27'

In [6]:
df_indice_atv_economica.head()

,ind_atv_econ,ano,mes,data_base
69,129.78,2009,10,2009-10-01
70,130.01,2009,11,2009-11-01
71,131.36,2009,12,2009-12-01
72,133.50,2010,1,2010-01-01
73,135.18,2010,2,2010-02-01


---

### Exógena - Estrutura Termo Taxa Juros
* Carregar planilha
* Filtrar colunas relevantes, renomeando quando necessário
* Dados informados possuem previsão que devem ser descartados.
* Dados anteriores a Out/2009 também serão descartados

In [7]:
# ParamETTJ
df_param_ettj = pd.read_excel(ARQUIVO_PARAMETROS_ESTRUTURA_TERMO_TX_JUROS)
df_param_ettj = df_param_ettj[['A_Ji','A_TJLP','A_INFLA_FINAME_OEC_MPME','A_INFLA_FINAME_OEC_Grande','A_PRE_FINAME_OEC_MPME','A_PRE_FINAME_OEC_Grande','A_PRE_12','A_PRE_24', 'A_PRE_36', 'A_PRE_60', 'Ano', 'Mês']]
df_param_ettj.rename(columns={'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_param_ettj = df_param_ettj.astype({'mes': int, 'ano': int})
df_param_ettj['data_base'] = df_param_ettj.ano.map(str) + '-' + df_param_ettj.mes.map("{:02}".format) + '-01'
df_param_ettj = df_param_ettj[(df_param_ettj.data_base >= '2009-10-01') & (df_param_ettj.data_base < DATA_CORRENTE)]

In [8]:
df_param_ettj.tail()

,A_Ji,A_TJLP,A_INFLA_FINAME_OEC_MPME,A_INFLA_FINAME_OEC_Grande,A_PRE_FINAME_OEC_MPME,A_PRE_FINAME_OEC_Grande,A_PRE_12,A_PRE_24,A_PRE_36,A_PRE_60,ano,mes,data_base
211,3.03,NaN,5.182887,5.158427,9.829825,9.782689,6.986920,7.940970,8.345853,8.799348,2021,8,2021-08-01
212,3.28,NaN,5.668285,5.661218,10.521768,10.508397,7.998453,8.942889,9.373741,9.870852,2021,9,2021-09-01
213,3.54,NaN,5.830968,5.843292,11.125235,11.151130,8.729619,9.637906,10.057843,10.562139,2021,10,2021-10-01
214,3.83,NaN,6.209388,6.211565,11.765139,11.772366,9.849967,10.597666,10.854279,11.165322,2021,11,2021-11-01
215,4.10,NaN,5.286694,5.280961,10.584886,10.581147,11.981002,12.181146,11.963991,11.765139,2021,12,2021-12-01


---

### Endógena - Finame Ônibus e Caminhões
* Carregar planilha
* Filtrar colunas relevantes, renomeando quando necessário
* Dados informados possuem previsão que devem ser descartados.
* Dados anteriores a Out/2009 também serão descartados

In [9]:
df_finame_oec = pd.read_excel(ARQUIVO_INDICES_ONIBUS_CAMINHOES, 'Plan1')
df_finame_oec = df_finame_oec[['A_DESEM_OEC', 'A_DESEM_OEC_MPME', 'A_DESEM_OEC_Grande', 'A_SPREAD_BNDES_MPME', 'A_SPREAD_BNDES_Grande', 'A_CUSTO_BNDES_MPME','A_CUSTO_BNDES_grande', 'A_VEND_OEC', 'A_SFIXA_SELIC', 'A_APROV_OEC', 'A_APROV_OEC_MPME', 'A_APROV_OEC_Grande', 'Ano', 'Mês']]
df_finame_oec.rename(columns={'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_finame_oec.rename(columns={'A_DESEM_OEC': 'desembolso_total', 'A_DESEM_OEC_MPME': 'desembolso_mpme', 'A_DESEM_OEC_Grande': 'desembolso_grande'}, inplace=True)
df_finame_oec.rename(columns={'A_SPREAD_BNDES_MPME': 'spread_mpme', 'A_SPREAD_BNDES_Grande': 'spread_grande', 'A_CUSTO_BNDES_MPME': 'custo_mpme'}, inplace=True)
df_finame_oec.rename(columns={'A_CUSTO_BNDES_grande': 'custo_grande', 'A_VEND_OEC': 'vendas', 'A_SFIXA_SELIC': 'tx_fixa_selic'}, inplace=True)
df_finame_oec.rename(columns={'A_APROV_OEC': 'aprovacao_total', 'A_APROV_OEC_MPME': 'aprovacao_mpme', 'A_APROV_OEC_Grande': 'aprovacao_grande'}, inplace=True)
df_finame_oec['data_base'] = df_finame_oec.ano.map(str) + '-' + df_finame_oec.mes.map("{:02}".format) + '-01'
df_finame_oec = df_finame_oec[(df_finame_oec.data_base >= '2009-10-01') & (df_finame_oec.data_base < DATA_CORRENTE)]
df_finame_oec.dropna(subset=['desembolso_total'], inplace=True)

In [10]:
df_finame_oec.tail()

,desembolso_total,desembolso_mpme,desembolso_grande,spread_mpme,spread_grande,custo_mpme,custo_grande,vendas,tx_fixa_selic,aprovacao_total,aprovacao_mpme,aprovacao_grande,ano,mes,data_base
210,600.554013,325.667389,274.886624,1.0,1.1,NaN,NaN,13049.0,0.45,700.736306,383.799488,316.936818,2021,7,2021-07-01
211,674.373890,354.907705,319.466185,1.0,1.1,NaN,NaN,14282.0,0.45,823.176882,498.507464,324.669418,2021,8,2021-08-01
212,623.317271,389.959495,233.357776,1.0,1.1,NaN,NaN,12728.0,0.45,859.089811,523.693027,335.396784,2021,9,2021-09-01
213,733.985836,450.416064,283.569772,1.0,1.1,NaN,NaN,12285.0,0.45,945.739167,565.569456,380.169711,2021,10,2021-10-01
214,873.246943,512.547902,360.699041,1.0,1.1,NaN,NaN,11930.0,0.45,1197.217223,755.171106,442.046117,2021,11,2021-11-01


### Merge Datasets
* Criar um Dataframe único com os campos que serão utlizados ou pelo menos analisados pelos modelos

In [11]:
df_oec = pd.merge(df_finame_oec, df_indice_atv_economica, how= 'inner',on='data_base', suffixes=('','_y'))
df_oec.drop(df_oec.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
df_oec = pd.merge(df_oec, df_param_ettj, how= 'inner',on='data_base', suffixes=('','_y'))
df_oec.drop(df_oec.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)


In [12]:
df_oec.head()

,desembolso_total,desembolso_mpme,desembolso_grande,spread_mpme,spread_grande,custo_mpme,custo_grande,vendas,tx_fixa_selic,aprovacao_total,...,A_Ji,A_TJLP,A_INFLA_FINAME_OEC_MPME,A_INFLA_FINAME_OEC_Grande,A_PRE_FINAME_OEC_MPME,A_PRE_FINAME_OEC_Grande,A_PRE_12,A_PRE_24,A_PRE_36,A_PRE_60
0,1767.32,1388.5,349.2,3.0,3.0,4.0,4.0,13123.0,NaN,1817.492460,...,NaN,6.0,5.822867,5.923976,12.796206,12.904365,9.723852,11.376501,12.213310,12.824550
1,1740.60,1339.9,362.5,3.0,3.0,4.0,4.0,13219.0,NaN,1999.128439,...,NaN,6.0,5.886187,5.958156,13.062494,13.149982,9.969168,11.560462,12.282385,12.869093
2,1691.54,1259.6,392.2,3.0,3.0,4.0,4.0,15104.0,NaN,2626.210087,...,NaN,6.0,6.045451,6.220147,13.233439,13.393701,9.984419,11.687233,12.488141,13.121518
3,2344.65,1682.7,616.7,3.0,3.0,4.0,4.0,11569.0,NaN,2441.408982,...,NaN,6.0,6.046619,6.167138,13.156129,13.268781,10.341847,11.944138,12.657941,13.269531
4,1631.05,1160.7,423.0,3.0,3.0,4.0,4.0,9581.0,NaN,2460.988063,...,NaN,6.0,5.694336,5.769518,12.657825,12.730024,10.452661,11.913480,12.576683,13.181880


In [13]:
df_oec.columns

Index(['desembolso_total', 'desembolso_mpme', 'desembolso_grande',
       'spread_mpme', 'spread_grande', 'custo_mpme', 'custo_grande', 'vendas',
       'tx_fixa_selic', 'aprovacao_total', 'aprovacao_mpme',
       'aprovacao_grande', 'ano', 'mes', 'data_base', 'ind_atv_econ', 'A_Ji',
       'A_TJLP', 'A_INFLA_FINAME_OEC_MPME', 'A_INFLA_FINAME_OEC_Grande',
       'A_PRE_FINAME_OEC_MPME', 'A_PRE_FINAME_OEC_Grande', 'A_PRE_12',
       'A_PRE_24', 'A_PRE_36', 'A_PRE_60'],
      dtype='object')

---
### MPME
* Até o momento o porte do beneficiário não foi filtrado (MPME e Grande).
* Para nosso estudo vamos retirar as colunas relacionadas a Grande Porte.
* Renomear colunas para nomes mais sugestivos

In [14]:
df_oec_mpme = df_oec[['ano', 'mes', 'data_base','desembolso_mpme', 'spread_mpme', 'custo_mpme', 
                      'vendas','tx_fixa_selic', 'aprovacao_mpme',  'ind_atv_econ', 
                      'A_Ji', 'A_TJLP','A_INFLA_FINAME_OEC_MPME', 'A_PRE_FINAME_OEC_MPME',
                      'A_PRE_12', 'A_PRE_24', 'A_PRE_36', 'A_PRE_60' ]]
df_oec_mpme.rename(columns={'desembolso_mpme': 'desembolso', 
                            'spread_mpme': 'spread', 
                            'custo_mpme': 'custo_psi', 
                            'aprovacao_mpme': 'aprovacao', 
                            'A_INFLA_FINAME_OEC_MPME': 'inflacao',
                            'A_PRE_FINAME_OEC_MPME': 'tx_pre_prazo_medio'},                            
                   inplace=True)


C:\Users\gdutr\miniconda3\envs\forecast\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [15]:
df_oec_mpme.tail()

,ano,mes,data_base,desembolso,spread,custo_psi,vendas,tx_fixa_selic,aprovacao,ind_atv_econ,A_Ji,A_TJLP,inflacao,tx_pre_prazo_medio,A_PRE_12,A_PRE_24,A_PRE_36,A_PRE_60
141,2021,7,2021-07-01,325.667389,1.0,NaN,13049.0,0.45,383.799488,138.68000,2.99,NaN,4.778909,8.710928,6.439720,7.558042,8.027355,8.573779
142,2021,8,2021-08-01,354.907705,1.0,NaN,14282.0,0.45,498.507464,138.06000,3.03,NaN,5.182887,9.829825,6.986920,7.940970,8.345853,8.799348
143,2021,9,2021-09-01,389.959495,1.0,NaN,12728.0,0.45,523.693027,137.42000,3.28,NaN,5.668285,10.521768,7.998453,8.942889,9.373741,9.870852
144,2021,10,2021-10-01,450.416064,1.0,NaN,12285.0,0.45,565.569456,136.87000,3.54,NaN,5.830968,11.125235,8.729619,9.637906,10.057843,10.562139
145,2021,11,2021-11-01,512.547902,1.0,NaN,11930.0,0.45,755.171106,137.28061,3.83,NaN,6.209388,11.765139,9.849967,10.597666,10.854279,11.165322


#### Cálculo do Custo de Captação do Recurso, de acordo com a data base (MM/AA) 

In [16]:
df_oec_mpme['custo_bndes'] = df_oec_mpme.apply(lambda row: calcula_custo(row), axis=1)

C:\Users\gdutr\AppData\Local\Temp/ipykernel_11968/3679597691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oec_mpme['custo_bndes'] = df_oec_mpme.apply(lambda row: calcula_custo(row), axis=1)


In [17]:
df_oec_mpme.columns

Index(['ano', 'mes', 'data_base', 'desembolso', 'spread', 'custo_psi',
       'vendas', 'tx_fixa_selic', 'aprovacao', 'ind_atv_econ', 'A_Ji',
       'A_TJLP', 'inflacao', 'tx_pre_prazo_medio', 'A_PRE_12', 'A_PRE_24',
       'A_PRE_36', 'A_PRE_60', 'custo_bndes'],
      dtype='object')

#### Retirando colunas desnecessárias, já que o custo de captação foi calculado.

In [18]:
df_oec_mpme.drop([ 'spread','custo_psi', 'tx_fixa_selic', 'A_Ji', 'A_TJLP', 'inflacao',  ], axis=1, inplace=True)

C:\Users\gdutr\miniconda3\envs\forecast\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### Calcula a diferença entre o custo de captação com o BNDES e o custo de captação no Mercado

In [21]:
df_oec_mpme['dif_custo_bndes_mercado'] = df_oec_mpme['custo_bndes'] - df_oec_mpme['tx_pre_prazo_medio']

C:\Users\gdutr\AppData\Local\Temp/ipykernel_11968/3985059479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oec_mpme['dif_custo_bndes_mercado'] = df_oec_mpme['custo_bndes'] - df_oec_mpme['tx_pre_prazo_medio']


In [23]:
df_oec_mpme.tail()

,ano,mes,data_base,desembolso,vendas,aprovacao,ind_atv_econ,tx_pre_prazo_medio,A_PRE_12,A_PRE_24,A_PRE_36,A_PRE_60,custo_bndes,dif_custo_bndes_mercado
141,2021,7,2021-07-01,325.667389,13049.0,383.799488,138.68000,8.710928,6.439720,7.558042,8.027355,8.573779,9.310928,0.6
142,2021,8,2021-08-01,354.907705,14282.0,498.507464,138.06000,9.829825,6.986920,7.940970,8.345853,8.799348,10.429825,0.6
143,2021,9,2021-09-01,389.959495,12728.0,523.693027,137.42000,10.521768,7.998453,8.942889,9.373741,9.870852,11.121768,0.6
144,2021,10,2021-10-01,450.416064,12285.0,565.569456,136.87000,11.125235,8.729619,9.637906,10.057843,10.562139,11.725235,0.6
145,2021,11,2021-11-01,512.547902,11930.0,755.171106,137.28061,11.765139,9.849967,10.597666,10.854279,11.165322,12.365139,0.6


#### Exporta o arquivo CSV com os dados, filtrados, consolidados e manipulados.

In [24]:
df_oec_mpme.to_csv(ARQUIVO_OUTPUT_OEC_MPME, index=False, sep=';', decimal=',')